In [ ]:
"""Table Purpose: Calculate the length and width  of the orphan dendrites

Why useful: 
1) Save the restriction that gets the orphan dendrites
2) Finalize the width as the 95th percentile
3) Calculate the lengths of the skeletons






"""

In [1]:
import numpy as np
import datajoint as dj
import time
import pymeshfix
import os
import datetime
import calcification_Module as cm
from meshparty import trimesh_io

#for supressing the output
import os, contextlib
import pathlib
import subprocess

#for error counting
from collections import Counter

#for reading in the new raw_skeleton files
import csv
import pandas as pd
from tqdm import tqdm


from meshparty import trimesh_io

#for filtering
import math
from pykdtree.kdtree import KDTree

In [2]:
#setting the address and the username
dj.config['database.host'] = '10.28.0.34'
dj.config['database.user'] = 'celiib'
dj.config['database.password'] = 'newceliipass'
dj.config['safemode']=True
dj.config["display.limit"] = 20

schema = dj.schema('microns_ta3p100')
ta3p100 = dj.create_virtual_module('ta3p100', 'microns_ta3p100')


Connecting celiib@10.28.0.34:3306


In [31]:
@schema
class SignificantOrphanDendriteWidthLength(dj.Computed):
    definition="""
    -> ta3p100.DendriteWidth
    ---
    width           :float #95th pecentile of min width range for final width
    skeleton_length :float #total length of skeleton
    """
    
    #calculating the significant orphan components
    start_table = (ta3p100.CompartmentOrphan.ComponentOrphan() & "compartment_type='Dendrite'").proj("n_vertex_indices")
    seg_ids = start_table.fetch("segment_id")
    len(seg_ids),len(set(seg_ids))

    component_total = []
    for s in set(seg_ids):
        seg_data = (start_table & "segment_id="+str(s)).fetch("component_index","n_vertex_indices")

        components = seg_data[0]
        n_vertices = seg_data[1]


        #get the maximum n_vertices 
        n_verts_max = max(n_vertices)
        components_max = components[n_vertices == n_verts_max][0]


        new_dict = dict(segment_id=s,component_index=components_max,n_vertex_indices=n_verts_max)
        component_total.append(new_dict)
    
    key_source = (ta3p100.DendriteWidth() & "min_width_max>0" & ta3p100.CompartmentOrphan.ComponentOrphan.proj() & "compartment_type='Dendrite'" & component_total)

    def make(self, key):
        #with the key pull down the width
        width = (ta3p100.DendriteWidth & key).fetch1("min_width_perc_95")
        
        #*****pull down the skeleton for the mesh (should be from highly discretized list****
        skeleton_edges = (ta3p100.FilteredSkeleton() & key).fetch1("edges")
        skeleton_edges.shape

        column_1 = skeleton_edges[:,0,:]
        column_2 = skeleton_edges[:,1,:]

        skeleton_length = 0

        for a,b in zip(column_1,column_2):
            skeleton_length+= np.linalg.norm(a-b)
        
        #save to the database
        new_key = dict(key,width=width,skeleton_length=skeleton_length)
        
        self.insert1(new_key,skip_duplicates=True)
        

        
        
    
    

In [32]:
start_time = time.time()
SignificantOrphanDendriteWidthLength.populate(reserve_jobs=True)
print(f"Total time: {time.time()-start_time}")

Total time: 24.997781991958618


In [33]:
SignificantOrphanDendriteWidthLength() & 

segmentation segmentation id,segment_id segment id unique within each Segmentation,decimation_ratio,"compartment_type Basal, Apical, spine head, etc.",component_index Which sub-compartment of a certain label this is.,discrete_length the maximum size of a discretized segment of postsyn skeleton,width 95th pecentile of min width range for final width,skeleton_length total length of skeleton
2,648518346341352006,0.35,Dendrite,0,10,258.371,165043.0
2,648518346341352223,0.35,Dendrite,0,10,300.18,286752.0
2,648518346341353186,0.35,Dendrite,0,10,335.249,323571.0
2,648518346341353607,0.35,Dendrite,0,10,582.155,399485.0
2,648518346341353883,0.35,Dendrite,0,10,494.533,271387.0
2,648518346341355539,0.35,Dendrite,0,10,270.548,402027.0
2,648518346341356260,0.35,Dendrite,0,10,254.333,89000.8
2,648518346341357297,0.35,Dendrite,0,10,473.654,944500.0
2,648518346341359123,0.35,Dendrite,0,10,263.893,395839.0
2,648518346341359289,0.35,Dendrite,0,10,328.8,390254.0


In [28]:
key = dict(segmentation=2,segment_id=648518346341352006)
width = (ta3p100.DendriteWidth & key).fetch1("min_width_perc_95")

#get the skeleton
#*****pull down the skeleton for the mesh (should be from highly discretized list****
skeleton_edges = (ta3p100.FilteredSkeleton() & key).fetch1("edges")
skeleton_edges.shape

column_1 = skeleton_edges[:,0,:]
column_2 = skeleton_edges[:,1,:]

skeleton_length = 0

for a,b in zip(column_1,column_2):
    skeleton_length+= np.linalg.norm(a-b)
print(skeleton_length)




165043.1950033336


In [29]:
key = dict(segmentation=2,segment_id=648518346341377161)
width = (ta3p100.DendriteWidth & key).fetch1("min_width_perc_95")

#get the skeleton
#*****pull down the skeleton for the mesh (should be from highly discretized list****
skeleton_edges = (ta3p100.FilteredSkeleton & key).fetch1("edges")
skeleton_edges.shape

column_1 = skeleton_edges[:,0,:]
column_1 = skeleton_edges[:,1,:]

skeleton_length = 0

for a,b in zip(column_1,column_2):
    skeleton_length+= np.linalg.norm(a-b)
print(skeleton_length)



26107861.862928435


In [23]:
import math
import numpy as np
#practice calculating the euclidean distance
a = np.array([1,1,1])
b = np.array([4,5,6])

a = np.array([4,7,8])
b = np.array([10,6,5]) 

actual_distance = math.sqrt((a[0]-b[0])**2 + (a[1]-b[1])**2 + (a[2]-b[2])**2)
print(actual_distance)
dist = np.linalg.norm(a-b)
print(dist)

a = np.array([[[1,1,1],[4,5,6]],[[4,7,8],[10,6,5]]])

col_1 = a[:,0,:]
col_2 = a[:,1,:]
dist = np.linalg.norm(col_1-col_2)
for a,b in zip(col_1,col_2):
    print(np.linalg.norm(a-b))


6.782329983125268
6.782329983125268
7.0710678118654755
6.782329983125268
